In [1]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
#os.system("cp vocab_300_with_numbers.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/31_normalised_220k/checkpoint-6500").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import datasets
from datasets import load_dataset, load_metric, Audio, load_from_disk
test = load_from_disk("test_mapped")
train = load_from_disk("train220k_mapped")

In [3]:
speech, sample_rate = test[0]["input_values"], 16000
input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0]).lower()
transcription

'zahtjevima europske komisije u odnosu na direkdirektivu europskog parlamenta i vijeća od 5 srpnja 206 godine zahvaljujem se na podršci svima koji su raspravljali i na samomekraju vezano uz ratifikaciju istambolske konvencije'

In [4]:
labs = test[0]["labels"]
processor.decode(labs).lower()

'zahtjevima europske komisije u odnosu na direktivu europskog parlamenta i vijeća od 5. srpnja 206. godine zahvaljujem se na podršci svima koji su raspravljali i na samome kraju vezano uz ratifikaciju istambulske konvencije'

In [10]:
def decode_labels(labels):
    return processor.decode(labels).lower()
def transcribe(inputs):
    speech = inputs
    input_values = processor(speech, sampling_rate=16000, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription
# Transcribing train data:
transcriptions = list(map(transcribe, [train[i]["input_values"] for i in range(1000, 1010)]))
labels = list(map(decode_labels, [train[i]["labels"] for i in range(1000, 1010)]))

In [11]:
for t, l in zip(transcriptions, labels):
    print(t, "\nlabels:", l)

s druge strane činjenicu da su nam djeca sve više podložna pretilosti i izložena sve većim rizicima a s druge strane onda gledamo što radimo zapravo da bismo im omogućili kvalitetniji i zdraviji život od malih nogu i vidimo da tu 
labels: s druge strane činjenicu da su nam djeca sve više podložna pretilosti i izložena sve većim rizicima a s druge strane onda gledamo što radimo zapravo da bismo im omogućili kvalitetniji i zdraviji život od malih nogu i vidimo da tu
nam mladi u predadolescentskom i adolescentskom razdoblju najviše ranjivi i najviše dakle izloženi vanjskim utjecajima pa tako i opojnim drogama i naravno ova vaša ideja je svakako dobro došla i više porazmisliti o strukturiranju slobodnog vremena 
labels: nam mladi u predadolescentskom i adolescentskom razdoblju najviše ranjivi i najviše dakle izloženi vanjskim utjecajima pa tako i opojnim drogama i naravno ova vaša ideja je svakako dobro došla i više porazmisliti o strukturiranju slobodnog vremena
za vrijeme pet šest godina

In [7]:
transcriptions = list(map(transcribe, test["input_values"]))
labels = list(map(decode_labels, test["labels"]))

import pandas as pd
pd.DataFrame(data={
    "machine_generated_transcripts": transcriptions,
    "original_transcript": labels
}).to_csv("34_model31_220k_performance.csv", index=False)